In [ ]:
from osgeo import gdal, osr
import os
from tqdm import tqdm
import rasterio
from rasterio.merge import merge
import re
from osgeo import gdal

In [ ]:
# Define the source and target directories
source_directory = "../input/gitignore/ghsl"

# Check if the source directory exists
if not os.path.exists(source_directory):
    raise FileNotFoundError(f"Source directory {source_directory} does not exist.")

# Iterate over each folder in the source directory
for folder in tqdm(os.listdir(source_directory)):
    folder_path = os.path.join(source_directory, folder)
    
    # Check if the path is indeed a directory
    if os.path.isdir(folder_path):
        # Initialize an empty list to hold the reprojected datasets
        reprojected_datasets = []

        # Iterate over each file in the directory
        for file in os.listdir(folder_path):
            if file.endswith('.tif'):
                # Define the paths for the source and target files
                src_file_path = os.path.join(folder_path, file)

                # Set up the gdalwarp options
                options = gdal.WarpOptions(
                    srcSRS='ESRI:54009',
                    dstSRS='EPSG:4326',
                    resampleAlg='cubicspline',
                    format='GTiff'
                )

                output_dir = f"../output/reprojected/{folder}"
                # Check if the output directory exists, and create it if it doesn't
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

                # Perform the warp operation
                gdal.Warp(destNameOrDestDS=f"{output_dir}/{file}", srcDSOrSrcDSTab=src_file_path, options=options)

                print(f"Reprojection complete. Output saved to {output_dir}/{file}")

# No specific action is taken for .ovr files in this script; they are simply ignored


# Set the directory containing subdirectories of .tif files
output_dir = "../output/reprojected"

# Iterate through each subdirectory in the output directory
for folder_name in tqdm([f for f in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, f))]):
    folder_path = os.path.join(output_dir, folder_name)
    
    # List all TIF files in the directory
    tif_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tif')]
    
    # Continue only if there are TIF files in the directory
    if tif_files:
        # Read all the TIF files into rasterio
        src_files_to_mosaic = []
        for tif in tif_files:
            src = rasterio.open(tif)
            src_files_to_mosaic.append(src)
        
        # Merge function returns a single mosaic array and the transformation info
        mosaic, out_trans = merge(src_files_to_mosaic)
        
        # Copy the metadata
        out_meta = src.meta.copy()
        
        # Update the metadata with new dimensions, transform, and CRS
        out_meta.update({"driver": "GTiff",
                         "height": mosaic.shape[1],
                         "width": mosaic.shape[2],
                         "transform": out_trans,
                         "crs": src.crs})
        
        # Define the output file name as folder_name.tif in the output directory
        output_file = os.path.join(output_dir, f'{folder_name}.tif')
        
        # Write the mosaic raster to disk
        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(mosaic)
        
        print(f"Merged file created at {output_file}")

In [ ]:
import os
import rasterio
from rasterio.merge import merge
from tqdm import tqdm

# Set the directory containing subdirectories of .tif files
output_dir = "../output/reprojected"

# Iterate through each subdirectory in the output directory
for folder_name in tqdm([f for f in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, f))]):
    folder_path = os.path.join(output_dir, folder_name)
    
    # List all TIF files in the directory
    tif_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tif')]
    
    # Continue only if there are TIF files in the directory
    if tif_files:
        # Read all the TIF files into rasterio
        src_files_to_mosaic = []
        for tif in tif_files:
            src = rasterio.open(tif)
            src_files_to_mosaic.append(src)
        
        # Merge function returns a single mosaic array and the transformation info
        mosaic, out_trans = merge(src_files_to_mosaic)
        
        # Copy the metadata
        out_meta = src.meta.copy()
        
        # Update the metadata with new dimensions, transform, and CRS
        out_meta.update({"driver": "GTiff",
                         "height": mosaic.shape[1],
                         "width": mosaic.shape[2],
                         "transform": out_trans,
                         "crs": src.crs})
        
        # Define the output file name as folder_name.tif in the output directory
        output_file = os.path.join(output_dir, f'{folder_name}.tif')
        
        # Write the mosaic raster to disk
        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(mosaic)
        
        print(f"Merged file created at {output_file}")

In [4]:
# Define the directories
output_dir = "../output/reprojected"
reference_file = "../output/reprojected/ghs-built-h.tif"  # Adjust path as necessary

# Ensure the reference file exists
if not os.path.exists(reference_file):
    raise FileNotFoundError(f"Reference file {reference_file} does not exist.")

# Retrieve the properties of the reference file
with gdal.Open(reference_file) as ref:
    ref_proj = ref.GetProjection()
    ref_geotrans = ref.GetGeoTransform()
    # Extract resolution from geotransform (assuming square pixels)
    ref_resolution = (ref_geotrans[1], ref_geotrans[5])

# Iterate over each file in the output directory
for file_name in os.listdir(output_dir):
    if file_name.endswith('.tif') and file_name != os.path.basename(reference_file):
        file_path = os.path.join(output_dir, file_name)

        # Define the destination file path
        aligned_file_path = os.path.join(output_dir, f"aligned_{file_name}")

        # Set up the gdalwarp options to align to reference file
        options = gdal.WarpOptions(
            format='GTiff',
            dstSRS=ref_proj,
            xRes=ref_resolution[0],
            yRes=ref_resolution[1],
            targetAlignedPixels=True,
            resampleAlg='near'  # or another resampling method as needed
        )

        # Perform the warp operation to align to the reference grid
        gdal.Warp(destNameOrDestDS=aligned_file_path, srcDSOrSrcDSTab=file_path, options=options)
        print(f"Alignment complete. Output saved to {aligned_file_path}")

c:\Users\reube\anaconda3\envs\processor\Lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Alignment complete. Output saved to ../output/reprojected\aligned_water.tif
Alignment complete. Output saved to ../output/reprojected\aligned_ghs-smod.tif
Alignment complete. Output saved to ../output/reprojected\aligned_ghs-built-v.tif
Alignment complete. Output saved to ../output/reprojected\aligned_ghs-built-c.tif
Alignment complete. Output saved to ../output/reprojected\aligned_ghs-pop.tif
Alignment complete. Output saved to ../output/reprojected\aligned_ghs-built-s.tif


: 

In [1]:
import shutil

# Define the source file and the destination file
source_file = "../output/reprojected/ghs-built-h.tif"
destination_file = "../output/reprojected/aligned_ghs-built-h.tif"

# Copy the source file to the destination file
shutil.copyfile(source_file, destination_file)

'../output/reprojected/aligned_ghs-built-h.tif'